# *신경망 학습*

## 5. 학습 알고리즘 구현하기

- 신경망에서 가중치와 편향을 훈련데이터에 맞춰 조정하는 과정을 '학습이라고 하며 아래 절차를 따름

> ### 1단계 - 미니배치
> - 훈련데이터 중 일부를 무작위로 호출
> - 선별된 데이터를 미니배치라 하며, 미니배치의 손실함수 값을 줄이는 것이 목표임

> ### 2단계 - 기울기 산출
> - 각 가중치 매개변수의 손실함수로부터의 기울기 $\frac{\delta L}{\delta W}$산출
> - 여기서 기울기는 손실함수의 값을 가장 작게 하는 방향을 제시

> ### 3단계 - 매개변수 갱신
> - 가중치 매개변수를 기울기 방향으로 학습률 만큼 갱신

> ### 4단계 - 반복
> - 1~3단계를 반복

- 위와 같은 방법은 미니배치를 무작위로 선정하기 때문에 **확률적 경사 하강법** 또는 **SGD(Stochastic Gradient Descent)**라고 함
___

### 1) 2층 신경망 클래스 구현하기

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from data.functions import *
from data.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads


- 위 클래스가 사용하는 변수와 내용은 대부분 앞에서 다룬 함수로 표현 가능
- TwoLayerNet 클래스의 중요한 변수는 다음과 같음

변수 | 설명
:---  | :---
params| 신경망의 매개변수를 보관하는 딕셔너리 변수
| params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향
| params['W2']은 2번째 층의 가중치, params['b2']은 2번째 층의 편향
grad| 기울기 보관하는 딕셔너리 변수(numerical_gradient()메서드의 반환 값)
| grad['W1']은 1번째 층의 가중치의 기울기, grad['b1']은 1번째 층 편향의 기울기
| grad['W2']은 2번째 층의 가중치의 기울기, grad['b2']은 2번째 층 편향의 기울기

- TwoLayerNet 클래스의 메서드는 다음과 같음

메서드|설명
---|---
\__init__(self, input_size, hidden_size, output_size)| 초기화를 수행하며 인수는 입력층 뉴런 수,은닉층 뉴런 수, 출력층 뉴런 수
predict(self, x)| 예측을 수행
| 인수 x 는 이미지 데이터
loss(self, x, t)| 손실 함수의 값을 계산
| 인수 x는 이미지 데이터, t는 정답 레이블
accuracy(self, x, t)|정확도 계산
numerical_gradient(self, x, t)|가중치 매개변수의 기울기
gradient(self, x, t)| 가중치 매개변수의 기울기
| numerical_gradient()의 성능 개선판!

In [2]:
net = TwoLayerNet(784, 100, 10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


- params에는 \__init__으로 생성한 임의의 가중치 행렬 보유 (np.random 함수에 init_weight의 곱)
- params에는 \__init__으로 생성한 0으로 이루어진 편향 행렬 보유

In [3]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)

grads['W1']

array([[ 6.41954490e-06,  4.77900386e-06, -2.41773008e-04, ...,
         1.52058761e-04,  9.21931465e-05,  2.27693209e-04],
       [-1.40050282e-05,  1.04347619e-05, -2.19735017e-04, ...,
         1.10293426e-04,  2.45337084e-05,  1.41514180e-04],
       [-2.41551068e-05, -5.93987015e-05, -2.41009575e-04, ...,
         5.65966163e-05,  2.44026310e-05,  2.66166398e-04],
       ...,
       [-2.29743846e-05, -1.50078645e-04, -9.71062653e-05, ...,
         1.30129618e-05, -1.43966903e-04,  9.72136061e-05],
       [ 9.50432844e-06, -3.50721141e-05, -8.10257417e-05, ...,
         6.69286782e-05, -3.47260887e-06,  5.87795612e-05],
       [-1.63335301e-05, -5.94519856e-05, -1.42146124e-04, ...,
         5.64283553e-05, -5.55379054e-05,  1.29453983e-04]])

- numerical_gradient 함수는 입력값과 레이블을 인수로 입력
- 입력값과 레이블로 cross_entropy_error loss 함수를 구현
- 가중치 값과 편향을 바탕으로 loss 함수값을 수치미분하여 기울기 산출
___

### 2) 미니배치 학습 구현하기

- 미니배치를 활용하여 신경망 학습 구현

In [4]:
import numpy as np
from data.mnist import load_mnist
from data.two_layer_net import TwoLayerNet

In [6]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)

train_loss_list = []

#하이퍼파라미터
iters = 100
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
network = TwoLayerNet(input_size = 784, hidden_size= 50, output_size=10)

for i in range(iters):
    #미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    #grad = network.gradient(x_batch, t_batch) #성능 개선판!
    
    #매개변수 계산
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    #학습경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i% 10 ==0:
        print(str(i) + 'th iter')

0th iter
10th iter
20th iter
30th iter
40th iter
50th iter
60th iter
70th iter
80th iter
90th iter


- 미니배치 크기는 100으로 총 60,000개의 훈련데이터에서 임의의 100개 데이터 호출
- 100개의 미니배치 대상으로 경사하강법을 수행해 매개변수를 갱신 (learning rate * gradient)
- 위 과정을 10,000번 반복
- 갱신할 때마다 손실함수를 계산하여 변화 추이 파악
![](image/fig 4-11.png)
___

### 3) 시험 데이터로 평가하기

- 오버피팅 방지를 위해 시험 데이터로 정확도를 평가
- 통상적으로 1에폭별로 훈련데이터와 시험데이터에 대한 정확도를 기록 (에폭 = 사용데이터수 / 훈련데이터수)
- 이를 위해 코드 일부 수정

In [7]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []


#하이퍼파라미터
iters = 100
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
network = TwoLayerNet(input_size = 784, hidden_size= 50, output_size=10)


#1에폭 당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)


for i in range(iters):
    #미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    #grad = network.gradient(x_batch, t_batch) #성능 개선판!
    
    #매개변수 계산
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    #학습경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    #에폭 당 정확도 계산
    if i % iter_per_epoch ==0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('train acc, test acc on ' +str(i)+ '-iter | ' + str(train_acc) + ', ' + str(test_acc))

train acc, test acc on 0-iter | 0.09751666666666667, 0.0974
